In [1]:
%load_ext autoreload
%autoreload 2

import math
from itertools import product
import uproot
import awkward
import numpy as np
from scipy.stats import binned_statistic_2d
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import matplotlib
matplotlib.rcParams.update({'font.size': 12})
import ROOT
from histogram import *
ROOT.gStyle.SetOptFit(1)

Welcome to JupyROOT 6.18/04


In [2]:
branches = [
    'run',
    'sub',
    'evt',
    'fileno',
    #'n_pfp',

    'backtracked_purity',
    'backtracked_completeness',
    'backtracked_process_is_stopping',
    'backtracked_end_in_tpc',
    
    'backtracked_pc_q_u',
    'backtracked_pc_q_v',
    'backtracked_pc_q_y',
    'backtracked_c_u',
    'backtracked_c_v',
    'backtracked_c_y',
    
    'backtracked_theta',
    'backtracked_phi',
    
    'trkhit_charge_u',
    'trkhit_charge_v',
    'trkhit_charge_y',
    'trkhit_channel_u',
    'trkhit_channel_v',
    'trkhit_channel_y',
    
    'allhit_charge_u',
    'allhit_charge_v',
    'allhit_charge_y',
    'allhit_channel_u',
    'allhit_channel_v',
    'allhit_channel_y',
    
    'calohit_charge_u',
    'calohit_charge_v',
    'calohit_charge_y',
    'calohit_channel_u',
    'calohit_channel_v',
    'calohit_channel_y',
    
    'areahit_charge_u',
    'areahit_charge_v',
    'areahit_charge_y',
    'areahit_channel_u',
    'areahit_channel_v',
    'areahit_channel_y',
    
    'sumhit_charge_u',
    'sumhit_charge_v',
    'sumhit_charge_y',
    'sumhit_channel_u',
    'sumhit_channel_v',
    'sumhit_channel_y',
]

In [3]:
# file_bnb = '/sbnd/data/users/gputnam/NuMuReco/bnblike_single_calo/sbnd_proton.root'
filetype = "proton"
file_bnb= "./sbnd_" + filetype + "_gaus.root"
folder = "caloana"
tree = "CalorimetryAnalyzer"

data = uproot.open(file_bnb)[folder][tree].arrays(branches, namedecode="utf-8")


In [4]:
hittypes = ["all", "trk", "area", "sum", "calo"]
planes = ["_u", "_v", "_y"]
constants = np.array([0.0198,0.0198,0.0198])
toexpand = ["backtracked_theta", "backtracked_phi", "backtracked_process_is_stopping", "backtracked_end_in_tpc",
           "evt", "fileno"]

for no,p in enumerate(planes):
    maxchannel = np.max(data["backtracked_c" + p].flatten())
    minchannel = np.min(data["backtracked_c" + p].flatten())
    channel_arr = np.stack(tuple([np.arange(minchannel, maxchannel+1) for _ in range(len(data["evt"]))]))

    data["allchannel_true_charge" + p] = np.stack(tuple([np.zeros((maxchannel-minchannel+1)) for i 
                                                           in range(len(data["evt"]))]))
    
    toset = []
    for i in range(len((data["backtracked_c"+p]))):
        data["allchannel_true_charge" + p][i, data["backtracked_c"+p][i]-minchannel] = data["backtracked_pc_q"+p][i]
    
    for h in hittypes:
        data[h+"_nhit"+p] = data[h+"hit_channel"+p]._stops - data[h+"hit_channel"+p]._starts
        allchannel = "allchannel_" + h + "charge" + p
        data[allchannel] = np.stack(tuple([np.zeros((maxchannel-minchannel+1)) for i 
                                                           in range(len(data["evt"]))]))


        for i in range(len(data[h+"hit_channel"+p])):
            data[allchannel][i, data[h+"hit_channel"+p][i]-minchannel] = data[h+"hit_charge"+p][i] / constants[no]
            
            
        data[h+"_qdiff"+p] = (np.sum(data[allchannel], axis=1) - np.sum(data["allchannel_true_charge" + p], axis=1)) / np.sum(data["allchannel_true_charge" + p], axis=1)

        goodchannel = data["allchannel_true_charge" + p] > 50000.
        data[h+"_hit_frac"+p] = np.sum((data[allchannel] > 1e-3) & goodchannel, axis=1) / np.sum(goodchannel, axis=1)
        data[h+"_channel_qdiff"+p] = (data[allchannel][goodchannel] - data["allchannel_true_charge" + p][goodchannel]) / data["allchannel_true_charge" + p][goodchannel]
        data[h+"_channel"+p] = channel_arr[goodchannel]
        repeat = np.sum(goodchannel, axis=1)
        for name in toexpand:
            data[h+"_"+name+p] = np.repeat(data[name], repeat)
        

        del data["allchannel_" + h + "charge" + p]
        del goodchannel
        
    del data["allchannel_true_charge" + p]
         


        

/home/grayputnam/.local/lib/python3.6/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in true_divide


In [5]:
save=False
outputdir=""

In [ ]:
lo = -0.4
hi = 0.4
h_dq_plane = PlaneHistogram(MultiHistogram(
                        ROOT.TH1D("diffQ_calo", "Calorimetric Charge", 100, lo, hi), 
                            ROOT.TH1D("diffQ_trk", "Tracked Charge", 100, lo, hi),
                            ROOT.TH1D("diffQ_all", "All Charge", 100, lo, hi),
                                           name=None)
                           , DEFAULT_PLANES[:-2])

when = (data['backtracked_end_in_tpc']) & (data['backtracked_process_is_stopping'])
# when = data['backtracked_process_is_stopping']
h_dq_plane.Fill(data, "calo_qdiff", "trk_qdiff", "all_qdiff",
                when=when,
                is_planar=None)
h_dq_plane.DrawEach()
h_dq_plane.XTitle("Q_{reco}-Q_{true}/Q_{true}")
h_dq_plane.YTitle("Entries")

if save: h_dq_plane.SaveEachAs(outputdir + "diffQ_gauss_comp.pdf")